<a href="https://colab.research.google.com/github/jmarcosbs/etl-de-usuarios-em-api/blob/main/Realizando_ETL_para_alterar_usu%C3%A1rios_em_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [2]:
# Utilize sua própria URL se quiser ;)
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [4]:
import pandas as pd

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[3893, 3900]


In [14]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 3893,
    "name": "Usuario 3",
    "account": {
      "id": 4128,
      "number": "78455-1",
      "agency": "0055",
      "balance": 1000.0,
      "limit": 1000.0
    },
    "card": {
      "id": 3785,
      "number": "9827 7723 2732 2918",
      "limit": 1000.0
    },
    "features": [
      {
        "id": 1220,
        "icon": "icon.png",
        "description": "string"
      }
    ],
    "news": [
      {
        "id": 238,
        "icon": "string",
        "description": "string"
      }
    ]
  },
  {
    "id": 3900,
    "name": "Usuario Novo",
    "account": {
      "id": 4135,
      "number": "1984-2",
      "agency": "48934",
      "balance": 234.0,
      "limit": 234.0
    },
    "card": {
      "id": 3792,
      "number": "9928 5621 4563 1456",
      "limit": 235.0
    },
    "features": [
      {
        "id": 1224,
        "icon": "string.png",
        "description": "string"
      }
    ],
    "news": [
      {
        "id": 7737,
        "icon": "string.

## **T**ransform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [15]:
print("---- Realizar alteração de nome de usuário ----")
print("Lista de usuários cadastrados:")

for user in users:
  print(f"ID: {user['id']} | Nome: {user['name']}")

usuario_editado = False

qual_user = int(input('Digite o ID do usuário que deseja alterar o nome: '))

while usuario_editado == False:

  for user in users:
    if user['id'] == qual_user:
        troca_user = input("Qual nome do usuário deseja colocar? ")
        user['name'] = troca_user
        usuario_editado = True

  if usuario_editado == True:

    print(f"Usuário {user['name']} editado com sucesso!")

  else:

    qual_user = int(input('Usuário não encontrado! Digite novamente o ID do Usuário: '))


---- Realizar alteração de nome de usuário ----
Lista de usuários cadastrados:
ID: 3893 | Nome: Usuario 3
ID: 3900 | Nome: Usuario Novo
Digite o ID do usuário que deseja alterar o nome: 6
Usuário não encontrado! Digite novamente o ID do Usuário: 5
Usuário não encontrado! Digite novamente o ID do Usuário: 3893
Qual nome do usuário deseja colocar? Novo Usuario
Usuário Usuario Novo editado com sucesso!


## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [13]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Usuario 3 updated? True!
User Usuario Novo updated? True!
